In [20]:
import keras
import keras.datasets
from keras.datasets import fashion_mnist
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from keras.utils import to_categorical

In [21]:
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split

In [22]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [23]:
print(x_train.shape, x_test.shape)
x_train = x_train/255
x_test = x_test/255
x_train = x_train.reshape( -1, 28,28,1)
x_test = x_test.reshape( -1, 28,28,1)
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
print(x_train.shape, x_test.shape, y_train_hot.shape, y_test_hot.shape)


(60000, 28, 28) (10000, 28, 28)
(60000, 28, 28, 1) (10000, 28, 28, 1) (60000, 10) (10000, 10)


In [24]:
batch_size = 1024
epochs =20
num_classes = 10

In [25]:
def BuildModel1(input_shape=(28,28,1), convSize=(3,3), PoolSize=(2,2)):
    modelFM = Sequential()
    modelFM.add(Conv2D(32, kernel_size=convSize,
                       activation='linear',
                       input_shape=input_shape,padding="same"))
    modelFM.add(LeakyReLU(alpha=0.1))
    modelFM.add(MaxPooling2D((2, 2),padding='same'))
    modelFM.add(Conv2D(64, convSize, 
                       activation='linear',
                       padding='same'))
    modelFM.add(LeakyReLU(alpha=0.1))
    modelFM.add(MaxPooling2D(pool_size=PoolSize,padding='same'))
    modelFM.add(Conv2D(128, convSize, 
                       activation='linear',
                       padding='same'))
    modelFM.add(LeakyReLU(alpha=0.1))
    modelFM.add(MaxPooling2D(pool_size=PoolSize,padding='same'))
    modelFM.add(Flatten())
    modelFM.add(Dense(128, activation='linear'))
    modelFM.add(LeakyReLU(alpha=0.1))
    modelFM.add(Dense(num_classes, activation='softmax'))
    return modelFM

In [26]:
modelFM = BuildModel1()
modelFM.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [27]:
modelFM.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 7, 7, 128)         73856     
__________

In [28]:
#(x_train, y_train), (x_test, y_test)
tr_x,cv_x,tr_y,cv_y=train_test_split(x_train,y_train_hot, test_size =0.4,random_state=30 );
tr_xx,cv_xx,tr_yx,cv_yx=train_test_split(tr_x,tr_y, test_size =0.4,random_state=30 );
tr_xxx,cv_xxx,tr_yxx,cv_yxx=train_test_split(tr_xx,tr_yx, test_size =0.4,random_state=30 );
tr_xxx1,cv_xxx1,tr_yxx1,cv_yxx1=train_test_split(tr_xxx,tr_yxx, test_size =0.2,random_state=30 );



model_train = modelFM.fit(tr_xxx1, tr_yxx1, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(cv_xxx1,cv_yxx1))

Train on 10368 samples, validate on 2592 samples
Epoch 1/20
10368/10368 [==============================] - 128s 12ms/step - loss: 1.7325 - acc: 0.4608 - val_loss: 0.9523 - val_acc: 0.6474
Epoch 2/20
10368/10368 [==============================] - 127s 12ms/step - loss: 0.8988 - acc: 0.6593 - val_loss: 0.7627 - val_acc: 0.7180
Epoch 3/20
10368/10368 [==============================] - 128s 12ms/step - loss: 0.7176 - acc: 0.7300 - val_loss: 0.6452 - val_acc: 0.7515
Epoch 4/20
10368/10368 [==============================] - 130s 13ms/step - loss: 0.6376 - acc: 0.7621 - val_loss: 0.6370 - val_acc: 0.7558
Epoch 5/20
10368/10368 [==============================] - 127s 12ms/step - loss: 0.5901 - acc: 0.7775 - val_loss: 0.5803 - val_acc: 0.7778
Epoch 6/20
10368/10368 [==============================] - 128s 12ms/step - loss: 0.5435 - acc: 0.7976 - val_loss: 0.5568 - val_acc: 0.7836
Epoch 7/20
10368/10368 [==============================] - 127s 12ms/step - loss: 0.5131 - acc: 0.8077 - val_loss: 0.4

In [29]:
test_eval = modelFM.evaluate(cv_xxx, cv_yxx, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

8640/8640 [==============================] - 33s 4ms/step
Test loss: 0.35058724684295833
Test accuracy: 0.8739583333333333


In [30]:
def BuildModelWithDO(input_shape=(28,28,1), convSize=(3,3), PoolSize=(2,2)):
    modelFM = Sequential()
    modelFM.add(Conv2D(32, kernel_size=convSize,
                       activation='linear',
                       input_shape=input_shape,padding="same"))
    modelFM.add(LeakyReLU(alpha=0.1))
    modelFM.add(MaxPooling2D((2, 2),padding='same'))
    modelFM.add(Dropout(0.1))    
    modelFM.add(Conv2D(64, convSize, 
                       activation='linear',
                       padding='same'))
    modelFM.add(LeakyReLU(alpha=0.1))
    modelFM.add(MaxPooling2D(pool_size=PoolSize,padding='same'))
    modelFM.add(Dropout(0.2))
    modelFM.add(Conv2D(128, convSize, 
                       activation='linear',
                       padding='same'))
    modelFM.add(LeakyReLU(alpha=0.1))
    modelFM.add(MaxPooling2D(pool_size=PoolSize,padding='same'))
    modelFM.add(Dropout(0.3))
    modelFM.add(Flatten())
    modelFM.add(Dense(128, activation='linear'))
    modelFM.add(LeakyReLU(alpha=0.1))
    modelFM.add(Dropout(0.3))
    modelFM.add(Dense(num_classes, activation='softmax'))
    return modelFM
modelFMDO = BuildModelWithDO()

In [31]:
modelFMDO.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
modelFMDO.summary()
model_trainDO = modelFMDO.fit(tr_xxx1, tr_yxx1, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(cv_xxx1,cv_yxx1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 7, 7, 64)          0         
__________

In [ ]:
accuracy = model_trainDO.history['acc']
val_accuracy = model_trainDO.history['val_acc']
loss = model_trainDO.history['loss']
val_loss = model_trainDO.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
predicted_classes = modelFMDO.predict(x_test)

In [ ]:
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

In [ ]:
test_eval = modelFMDO.evaluate(x_test, y_test_hot, verbose=1)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])